In [1]:
# necessary imports for the lab
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import IPython.display
import os
import numpy as np
import datetime
import sys

# 1. Object Localization

In [15]:
video = cv2.VideoCapture("original.mov")

while True:
    success, frame = video.read()
    
    if success:
        frame = cv2.resize(frame, (368, 640))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        
        _, th1 = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(th1, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        if not contours:
            cv2.imshow('Object Localization', frame)
        else:
            img = frame.copy()
            min_area_threshold = 50
            for contour in contours:
                area = cv2.contourArea(contour)
                if area > min_area_threshold:
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 3)        

            cv2.imshow('Object Localization', img)
        
    else:
        break
        
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

# 2. Tracking

In [16]:
from tracker import *
tracker = EuclideanDistTracker()
video = cv2.VideoCapture("original.mov")
detections = []

while True:
    success, frame = video.read()

    if success:
        frame = cv2.resize(frame, (368, 640))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        _, th1 = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(th1, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        if not contours:
            cv2.imshow('Object Tracking', frame)
        else:
            img = frame.copy()
            min_area_threshold = 50
            for contour in contours:
                area = cv2.contourArea(contour)
                if area > min_area_threshold:
                    x, y, w, h = cv2.boundingRect(contour)
                    detections.append((x, y, w, h))
                    
            # Update tracker with the detections
            boxes_ids = tracker.update(detections)

            # Visualize bounding boxes and IDs
            for box_id in boxes_ids:
                x, y, w, h, id = box_id
                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 3)
                cv2.putText(img, str(id), (x, y + 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            cv2.imshow('Object Tracking', img)
            detections = []  # Reset detections for the next frame

    else:
        break

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

{0: (357, 5)}
{0: (355, 4), 1: (240, 3)}
{0: (355, 4), 1: (237, 8)}
{0: (355, 6), 1: (237, 8)}
{0: (355, 6), 1: (236, 14)}
{0: (348, 7), 1: (236, 14)}
{0: (348, 7), 1: (235, 19)}
{0: (348, 7), 1: (235, 19)}
{0: (348, 7), 1: (229, 25)}
{0: (343, 7), 1: (229, 25)}
{0: (343, 7), 1: (226, 31)}
{0: (343, 7), 1: (225, 43)}
{0: (347, 6), 1: (225, 43)}
{1: (226, 56), 0: (347, 6)}
{1: (226, 56), 0: (348, 5)}
{1: (228, 68), 0: (348, 5)}
{1: (228, 68), 0: (353, 5)}
{1: (230, 81), 0: (353, 5)}
{1: (230, 81), 0: (357, 4)}
{1: (232, 94), 0: (357, 4)}
{1: (235, 107)}
{1: (236, 120)}
{1: (238, 132)}
{1: (240, 145)}
{1: (242, 157)}
{1: (244, 170)}
{1: (246, 182)}
{1: (248, 195)}
{1: (250, 206)}
{1: (251, 219)}
{1: (253, 230)}
{1: (256, 241)}
{1: (257, 253)}
{1: (259, 266)}
{1: (262, 278)}
{1: (264, 288)}
{1: (266, 297)}
{1: (266, 307)}
{1: (264, 319)}
{1: (265, 330)}
{1: (267, 342)}
{1: (272, 352)}
{1: (278, 361)}
{1: (281, 370)}
{1: (285, 379)}
{1: (286, 388)}
{1: (286, 396)}
{1: (289, 404)}
{1: (293,

# Feature-Based tracking

In [2]:
def track(file_path, fps):
    # Open the video file
    cap = cv2.VideoCapture(file_path)


    # Parameters for Shi-Tomasi corner detection and Lucas-Kanade Optical Flow
    parameters_shitomasi = dict(maxCorners=100, qualityLevel=0.3, minDistance=10)
    parameter_lucas_kanade = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS |
                                                                          cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    colours = np.random.randint(0, 255, size=(100, 3))

    # Read the first frame and convert to grayscale
    ok, frame = cap.read()
    if not ok:
        print("[ERROR] cannot get frame from video")
        sys.exit()
    frame_gray_init = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect corners using Shi-Tomasi in the initial frame
    edges = cv2.goodFeaturesToTrack(frame_gray_init, mask=None, **parameters_shitomasi)

    canvas = np.zeros_like(frame)

    while True:
        ok, frame = cap.read()
        if not ok:
            print("[INFO] end of file reached")
            break
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow using Lucas-Kanade
        update_edges, status, errors = cv2.calcOpticalFlowPyrLK(frame_gray_init, frame_gray, edges, None,
                                                                **parameter_lucas_kanade)
        new_edges = update_edges[status == 1]
        old_edges = edges[status == 1]

        for i, (new, old) in enumerate(zip(new_edges, old_edges)):
            a, b = new.ravel()
            c, d = old.ravel()

            # Draw lines between old and new corner points with random colour
            mask = cv2.line(canvas, (int(a), int(b)), (int(c), int(d)), colours[i].tolist(), 2)
            # Draw circle around new position
            frame = cv2.circle(frame, (int(a), int(b)), 5, colours[i].tolist(), -1)

        result = cv2.add(frame, mask)
        cv2.imshow('Optical Flow (sparse)', result)
        if cv2.waitKey(fps) & 0xFF == ord('q'):
            break
        frame_gray_init = frame_gray.copy()
        edges = new_edges.reshape(-1, 1, 2)

    cv2.destroyAllWindows()
    cap.release()

In [ ]:
track("Easy.mp4", 30)

In [3]:
track("original.mov", 30)

[INFO] end of file reached
